# Analysis findings

+ Based on Gender Demographics, males account for 81.15% of all players that purchase.
+ Based on Purchasing Analysis, males account for the majority (81.68%) of total purchase value.
  Females account for 16.74% of total purchase value.
  This might indicate that there is a correlation between gender and purchases
+ Most players are between 15 and 29 years old (77.83%) while most players are between 20 and 24 years old (45.20%)
+ Players between 20 and 24 spend in total most on purchases (USD 978.77)
  Per player however, players between 20 and 24 spend least on items (USD 3.78).
  Players being 40 years old or older spend least in total, however per player they spend most (USD 4.89)
+ With one exception (USD 17.06), no one spends more then USD 17.00 on items. 
+ There seems to be no correlation between Most Popular Items and Most Profitable Items.
  There's only one item (Retribution Axe) that appears in both top 5 lists.
  

In [2]:
# Import Dependencies
import pandas as pd
import os

In [3]:
# Load data into dataframe
input_file = input("Enter the name of the JSON file you want to analyze (without extension): ") + ".JSON"
# Set file path (input file should be located on the same level as the folder raw_data)
filepath = os.path.join('raw_data', input_file)
purchases_df = pd.read_json(filepath)

Enter the name of the JSON file you want to analyze (without extension): purchase_data


## Player Count

In [4]:
# Total Number of Players
tot_players = len(purchases_df["SN"].unique())
tot_players_df = pd.DataFrame({"Total Players":[tot_players]})

tot_players_df

,Total Players
0,573


## Purchasing Analysis (Total)

In [5]:
# Calculations
total_unique_items = len(purchases_df["Item ID"].unique())
avg_purchase_price = purchases_df["Price"].mean()
total_purchases = len(purchases_df)
total_revenue = purchases_df["Price"].sum().mean()
# create dataframe
purchase_analysis_df = pd.DataFrame({"Number of Unique Items":[total_unique_items],
                                     "Average Price":[avg_purchase_price],
                                     "Number of Purchases":[total_purchases],
                                     "Total Revenue":[total_revenue]})
# Apply formatting
purchase_analysis_df["Average Price"] = purchase_analysis_df["Average Price"].map("${0:,.2f}".format)
purchase_analysis_df["Total Revenue"] = purchase_analysis_df["Total Revenue"].map("${0:,.2f}".format)
# Rearrange columns
columnsTitles = ["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"]
purchase_analysis_df = purchase_analysis_df.reindex(columns=columnsTitles)

purchase_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


## Gender Demographics

In [7]:
# Calculations
gender_count = purchases_df.groupby(["Gender"])["SN"].nunique()
gender_perc = 100*gender_count/tot_players
# Create dataframe
gender_demographics_df = pd.DataFrame({"Total Count":gender_count,"Percentage of Players":gender_perc})
# Apply formatting
gender_demographics_df["Percentage of Players"] = gender_demographics_df["Percentage of Players"].map("{0:,.2f}".format)
# Rearrange columns
columnsTitles = ["Percentage of Players", "Total Count"]
gender_demographics_df = gender_demographics_df.reindex(columns=columnsTitles)
# Apply sorting
gender_demographics_df= gender_demographics_df.sort_values("Total Count", ascending = False, inplace = False)
# Remove index header name
gender_demographics_df = gender_demographics_df.rename_axis(None)

gender_demographics_df

,Percentage of Players,Total Count
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


## Purchasing Analysis (Gender)

In [8]:
# Calculations
gender_purchase_total = purchases_df.groupby(["Gender"]).sum()["Price"]
gender_average = purchases_df.groupby(["Gender"]).mean()["Price"]
gender_counts = purchases_df.groupby(["Gender"]).count()["Price"]
normalized_total = gender_purchase_total / gender_demographics_df["Total Count"]
# Generate dataframe
purch_data_gender_df = pd.DataFrame({"Purchase Count": gender_counts,
                                     "Average Purchase Price": gender_average,
                                     "Total Purchase Value": gender_purchase_total,
                                     "Normalized Totals": normalized_total})
# Rearrange columns
columnsTitles = ["Purchase Count", "Average Purchase Price", "Total Purchase Value","Normalized Totals"]
purch_data_gender_df = purch_data_gender_df.reindex(columns=columnsTitles)
# Apply formatting
format_list = ["Average Purchase Price","Normalized Totals","Total Purchase Value"]
key_format = "${0:.2f}"
for key, value in purch_data_gender_df.items():
    if key in format_list:
        purch_data_gender_df[key] = value.map(key_format.format)

purch_data_gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,$1867.68,$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


## Age Demographics

In [10]:
# Define age bins and groups
bins = [0, 10, 15, 20, 25, 30, 35, 40, 150]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39", "40+"]
purchases_df["Age Group"] = pd.cut(purchases_df["Age"], bins, right = False, labels=group_names)
# Calculations
age_group_total = purchases_df.groupby(["Age Group"])["SN"].nunique()
age_group_perc = age_group_total/tot_players
# Create dataframe
age_group_df = pd.DataFrame({"Total Count": age_group_total, "Percentage of Players": 100*age_group_perc})
# Apply formatting
age_group_df["Percentage of Players"] = age_group_df["Percentage of Players"].map("{0:,.2f}".format)
# Rearrange columns
columnsTitles = ["Percentage of Players","Total Count"]
age_group_df = age_group_df.reindex(columns=columnsTitles)
# Remove index header name
age_group_df = age_group_df.rename_axis(None)

age_group_df

,Percentage of Players,Total Count
<10,3.32,19
10-14,4.01,23
15-19,17.45,100
20-24,45.20,259
25-29,15.18,87
30-34,8.20,47
35-39,4.71,27
40+,1.92,11


## Purchasing Analysis (Age Group)

In [11]:
# Calculations
unique_persons = purchases_df.groupby(["Age Group"])["SN"].nunique()
group_purch = purchases_df.groupby(["Age Group"])["Item ID"].count()
group_avg_price = purchases_df.groupby(["Age Group"])["Price"].mean().round(2)
group_total = purchases_df.groupby(["Age Group"])["Price"].sum()
normalized_total = group_total/unique_persons
# Create dataframe
purch_analysis_group_df = pd.DataFrame({"Purchase Count":group_purch,
                                        "Average Purchase Price":group_avg_price,
                                        "Total Purchase Value":group_total,
                                        "Normalized Totals":normalized_total})
# Rearrange columns
columnsTitles = ["Purchase Count","Average Purchase Price","Total Purchase Value","Normalized Totals"]
purch_analysis_group_df = purch_analysis_group_df.reindex(columns=columnsTitles)
# Apply formatting
format_list = ["Average Purchase Price","Normalized Totals","Total Purchase Value"]
key_format = "${0:.2f}"
for key, value in purch_analysis_group_df.items():
    if key in format_list:
        purch_analysis_group_df[key] = value.map(key_format.format)

purch_analysis_group_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
<10,28,$2.98,$83.46,$4.39
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-39,42,$2.84,$119.40,$4.42
40+,17,$3.16,$53.75,$4.89


## Top Spenders

In [13]:
# Calculations
total_spendings = purchases_df.groupby(["SN"])["Price"].sum()
total_buyers = purchases_df.groupby(["SN"])["Item ID"].count()
avg_spending = total_spendings/total_buyers
# Create dataframe
purch_analysis_df = pd.DataFrame({"Purchase Count":total_buyers,
                                  "Average Purchase Price":avg_spending,
                                  "Total Purchase Value":total_spendings})
# Rearrange columns
columnsTitles=["Purchase Count","Average Purchase Price","Total Purchase Value"]
purch_analysis_df=purch_analysis_df.reindex(columns=columnsTitles)
# Apply sorting
purch_analysis_df= purch_analysis_df.sort_values(["Total Purchase Value","Purchase Count"], ascending = [False, False])
# Apply formatting
purch_analysis_df["Total Purchase Value"] = purch_analysis_df["Total Purchase Value"].map("${0:,.2f}".format)
purch_analysis_df["Average Purchase Price"] = purch_analysis_df["Average Purchase Price"].map("${0:,.2f}".format)

# Top 5 spenders
purch_analysis_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


## Most Popular Items

In [14]:
# Create dataframe for item data
top_items = purchases_df.groupby(["Item ID", "Item Name","Price"])["Item ID"].count()
top_items_df = pd.DataFrame(top_items)
top_items_df = top_items_df.rename(columns={"Item ID":"Purchase Count"})
top_items_df.reset_index(inplace=True) #reset index as you can't merge on an index
# Create dataframe for purchases (revenue) data
total_revenue = purchases_df.groupby(["Item ID"])["Price"].sum()
total_revenue_df = pd.DataFrame(total_revenue)
total_revenue_df = total_revenue_df.rename(columns={"Price":"Total Purchase Value"})
total_revenue_df.reset_index(inplace=True) #reset index as you can't merge on an index
# Merge dataframes
merge_top_items_df = pd.merge(top_items_df, total_revenue_df, on="Item ID", how="inner")
merge_top_items_df.set_index(["Item ID", "Item Name"], inplace=True)
# Rename/rearrange columns
merge_top_items_df = merge_top_items_df.rename(columns={"Price":"Item Price"})
total_revenue_df.reset_index(inplace=True) 
columnsTitles=["Purchase Count","Item Price","Total Purchase Value"]
merge_top_items_df = merge_top_items_df.reindex(columns=columnsTitles)
# Create new dataframe so we can use current dataframe for Most Profitable Items Analysis
popularity_df = merge_top_items_df.copy(deep = True)
# Apply sorting
popularity_df.sort_values(["Purchase Count","Total Purchase Value"], ascending = [False, False], inplace = True)
# Apply formatting
popularity_df["Item Price"] = popularity_df["Item Price"].map("${0:,.2f}".format)
popularity_df["Total Purchase Value"] = popularity_df["Total Purchase Value"].map("${0:,.2f}".format)

#Top 5 popular items
popularity_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
34,Retribution Axe,9,$4.14,$37.26
31,Trickster,9,$2.07,$18.63
13,Serenity,9,$1.49,$13.41


## Most Profitable Items

In [15]:
# Apply sorting
merge_top_items_df.sort_values(["Total Purchase Value", "Purchase Count"], ascending = [False, False], inplace = True)
# Apply formatting
merge_top_items_df["Item Price"] = merge_top_items_df["Item Price"].map("${0:,.2f}".format)
merge_top_items_df["Total Purchase Value"] = merge_top_items_df["Total Purchase Value"].map("${0:,.2f}".format)

# Top 5 profitable items
merge_top_items_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88


In [12]:
print("ANALYSIS REPORT")
print('='*15)
print("Player Count")
display(tot_players_df)
print("Purchasing Analysis (Total)")
display(purchase_analysis_df)
print("Gender Demographics")
display(gender_demographics_df)
print("Purchasing Analysis (Gender)")
display(purch_data_gender_df)
print("Age Demographics")
display(age_group_df)
print("Purchasing Analysis (Age Group)")
display(purch_analysis_group_df)
print("Top Spenders")
display(purch_analysis_df.head())
print("Most Popular Items")
display(popularity_df.head())
print("Most Profitable Items")
display(merge_top_items_df.head())

ANALYSIS REPORT
Player Count


,Total Players
0,573


Purchasing Analysis (Total)


,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,$2.93,780,183,"$2,286.33"


Gender Demographics


...

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


Most Popular Items


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
34,Retribution Axe,9,$4.14,$37.26
31,Trickster,9,$2.07,$18.63
13,Serenity,9,$1.49,$13.41


Most Profitable Items


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
